In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import json
from statistics import mean

## **Data read**

## Enquiry data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
BASE_PATH = '/content/drive/MyDrive/Colab Notebooks/MonsoonCreditTech/senior_ds_test/'

Mounted at /content/drive


In [ ]:
f = open(BASE_PATH + '/data/train/enquiry_data_train.json')
enquiry_train_json = json.load(f)
print("enquiry_train_json = {}".format(len(enquiry_train_json)))
enquiry_train_list = [i for sublist in enquiry_train_json for i in sublist]
print("enquiry_train_list = {}".format(len(enquiry_train_list)))

# Get a dataframe
enquiry_train = pd.DataFrame.from_records(enquiry_train_list)
print(enquiry_train.shape, enquiry_train.columns, enquiry_train.uid.nunique())
enquiry_train.head(5)

enquiry_train_json = 261383
enquiry_train_list = 1909926
(1909926, 4) Index(['enquiry_type', 'enquiry_amt', 'enquiry_date', 'uid'], dtype='object') 261383


,enquiry_type,enquiry_amt,enquiry_date,uid
0,Interbank credit,168839,2020-11-08,AAA08065248
1,Mobile operator loan,268392,2020-09-20,AAA08065248
2,Mobile operator loan,36082,2020-06-19,AAA08065248
3,Interbank credit,180467,2019-10-22,AAA08065248
4,Cash loan (non-earmarked),227459,2020-05-24,AAA08065248


In [ ]:
# enquiry_train.describe()
print("enquiry_train: cred type = \n{}".format(enquiry_train.enquiry_type.unique()))

# missing values
print("\nenquiry_train missing values = \n{}".format(enquiry_train.isnull().sum()))

enquiry_train: cred type = 
['Interbank credit' 'Mobile operator loan' 'Cash loan (non-earmarked)'
 'Mortgage' 'Revolving loans'
 'Loan for purchase of shares (margin lending)' 'Cash loans' 'Car loan'
 'Another type of loan' 'Loan for the purchase of equipment'
 'Real estate loan' 'Credit card' 'Unknown type of loan' 'Microloan'
 'Consumer credit' 'Loan for working capital replenishment'
 'Loan for business development']

enquiry_train missing values = 
enquiry_type    0
enquiry_amt     0
enquiry_date    0
uid             0
dtype: int64


## Accounts data

In [ ]:
f = open(BASE_PATH + '/data/train/accounts_data_train.json')
accounts_train_json = json.load(f)
print("accounts_train_json = {}".format(len(accounts_train_json)))
accounts_train_list = [i for sublist in accounts_train_json for i in sublist]
print("accounts_train_list = {}".format(len(accounts_train_list)))

# Get a dataframe
accounts_train = pd.DataFrame.from_records(accounts_train_list)
print(accounts_train.shape, accounts_train.columns, accounts_train.uid.nunique())
accounts_train.head(5)

accounts_train_json = 223918
accounts_train_list = 1245310
(1245310, 7) Index(['credit_type', 'loan_amount', 'amount_overdue', 'open_date',
       'closed_date', 'payment_hist_string', 'uid'],
      dtype='object') 223918


,credit_type,loan_amount,amount_overdue,open_date,closed_date,payment_hist_string,uid
0,Consumer credit,272745.000,0.0,2018-09-22,2020-02-22,0000000000000000000000100000000000000000000000...,AAA09044550
1,Consumer credit,4500.000,0.0,2018-03-08,2019-07-25,000000000000000014044000000000000000000000000000,AAA09044550
2,Credit card,80996.445,0.0,2020-06-29,NaN,000000000000000000,AAA10545297
3,Consumer credit,43771.500,0.0,2020-06-09,2020-09-09,000000000,AAA14112888
4,Credit card,10480.500,0.0,2014-09-10,NaN,0000000000000000000000000000000000000000000000...,AAA20326915


In [ ]:
# accounts_train.describe()
print("accounts_train: cred type = \n{}".format(accounts_train.credit_type.unique()))

# missing values
print("\naccounts_train missing values = \n{}".format(accounts_train.isnull().sum()))

accounts_train: cred type = 
['Consumer credit' 'Credit card' 'Mortgage' 'Car loan'
 'Loan for business development' 'Microloan' 'Another type of loan'
 'Loan for working capital replenishment' 'Unknown type of loan'
 'Loan for the purchase of equipment' 'Real estate loan'
 'Cash loan (non-earmarked)'
 'Loan for purchase of shares (margin lending)' 'Interbank credit'
 'Mobile operator loan']

accounts_train missing values = 
credit_type                 0
loan_amount                 3
amount_overdue              0
open_date                   0
closed_date            463035
payment_hist_string         0
uid                         0
dtype: int64


In [ ]:
# accounts_train[accounts_train["loan_amount"].isnull()]
accounts_train = accounts_train[~(accounts_train["loan_amount"].isnull())]
print(accounts_train.isnull().sum())

credit_type                 0
loan_amount                 0
amount_overdue              0
open_date                   0
closed_date            463033
payment_hist_string         0
uid                         0
dtype: int64


In [ ]:
def wrap(s, w):
    pay_hist_list = [s[i:i + w] for i in range(0, len(s), w)]
    pay_hist_list = [i for i in pay_hist_list if i != '000']
    pay_hist_list = [int(i) for i in pay_hist_list]
    if len(pay_hist_list) == 0:
        return 0
    return mean(pay_hist_list)

# test_accounts_train_sample = test_accounts_train.sample(2)
accounts_train["decode_payment_hist_string"] = accounts_train.apply(lambda x: wrap(x["payment_hist_string"], 3), axis=1)
accounts_train.head(2)

,credit_type,loan_amount,amount_overdue,open_date,closed_date,payment_hist_string,uid,decode_payment_hist_string
0,Consumer credit,272745.0,0.0,2018-09-22,2020-02-22,0000000000000000000000100000000000000000000000...,AAA09044550,10.0
1,Consumer credit,4500.0,0.0,2018-03-08,2019-07-25,000000000000000014044000000000000000000000000000,AAA09044550,29.0


In [ ]:
accounts_train['open_date'] = pd.to_datetime(accounts_train['open_date'])
accounts_train['closed_date'] = pd.to_datetime(accounts_train['closed_date']) 

In [ ]:
accounts_train["closed_date"].fillna(pd.to_datetime('today').strftime("%Y-%m-%d"), inplace=True)

In [ ]:
# accounts_train[accounts_train.closed_date.isnull()]
accounts_train.head()

,credit_type,loan_amount,amount_overdue,open_date,closed_date,payment_hist_string,uid,decode_payment_hist_string
0,Consumer credit,272745.000,0.0,2018-09-22,2020-02-22,0000000000000000000000100000000000000000000000...,AAA09044550,10.0
1,Consumer credit,4500.000,0.0,2018-03-08,2019-07-25,000000000000000014044000000000000000000000000000,AAA09044550,29.0
2,Credit card,80996.445,0.0,2020-06-29,2022-05-02,000000000000000000,AAA10545297,0.0
3,Consumer credit,43771.500,0.0,2020-06-09,2020-09-09,000000000,AAA14112888,0.0
4,Credit card,10480.500,0.0,2014-09-10,2022-05-02,0000000000000000000000000000000000000000000000...,AAA20326915,467.0


In [ ]:
accounts_train["loan_period"] = accounts_train['closed_date'] - accounts_train['open_date']
accounts_train.head()

,credit_type,loan_amount,amount_overdue,open_date,closed_date,payment_hist_string,uid,decode_payment_hist_string,loan_period
0,Consumer credit,272745.000,0.0,2018-09-22,2020-02-22,0000000000000000000000100000000000000000000000...,AAA09044550,10.0,518 days
1,Consumer credit,4500.000,0.0,2018-03-08,2019-07-25,000000000000000014044000000000000000000000000000,AAA09044550,29.0,504 days
2,Credit card,80996.445,0.0,2020-06-29,2022-05-02,000000000000000000,AAA10545297,0.0,672 days
3,Consumer credit,43771.500,0.0,2020-06-09,2020-09-09,000000000,AAA14112888,0.0,92 days
4,Credit card,10480.500,0.0,2014-09-10,2022-05-02,0000000000000000000000000000000000000000000000...,AAA20326915,467.0,2791 days


In [ ]:
accounts_train_save = accounts_train.drop(columns=["open_date", "closed_date", "payment_hist_string"])
print(accounts_train_save.dtypes)
accounts_train_save.head(2)

credit_type                            object
loan_amount                           float64
amount_overdue                        float64
uid                                    object
decode_payment_hist_string            float64
loan_period                   timedelta64[ns]
dtype: object


,credit_type,loan_amount,amount_overdue,uid,decode_payment_hist_string,loan_period
0,Consumer credit,272745.0,0.0,AAA09044550,10.0,518 days
1,Consumer credit,4500.0,0.0,AAA09044550,29.0,504 days


In [ ]:
accounts_train_save = pd.merge(accounts_train_save, flag_train, left_on=["uid"], right_on=["uid"])
accounts_train_save.head()

,credit_type,loan_amount,amount_overdue,uid,decode_payment_hist_string,loan_period,NAME_CONTRACT_TYPE,TARGET
0,Consumer credit,272745.000,0.0,AAA09044550,10.0,518 days,Cash loans,0
1,Consumer credit,4500.000,0.0,AAA09044550,29.0,504 days,Cash loans,0
2,Credit card,80996.445,0.0,AAA10545297,0.0,672 days,Cash loans,0
3,Consumer credit,43771.500,0.0,AAA14112888,0.0,92 days,Cash loans,0
4,Credit card,10480.500,0.0,AAA20326915,467.0,2791 days,Cash loans,0


In [ ]:
from sklearn import preprocessing
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
print("before encoding = {}".format(accounts_train_save['credit_type'].unique()))

# Encode labels in column 'species'.
accounts_train_save['credit_type']= label_encoder.fit_transform(accounts_train_save['credit_type'])
print("after encoding = {}".format(accounts_train_save['credit_type'].unique()))

before encoding = ['Consumer credit' 'Credit card' 'Mortgage' 'Car loan'
 'Loan for business development' 'Microloan' 'Another type of loan'
 'Loan for working capital replenishment' 'Unknown type of loan'
 'Loan for the purchase of equipment' 'Real estate loan'
 'Cash loan (non-earmarked)'
 'Loan for purchase of shares (margin lending)' 'Interbank credit'
 'Mobile operator loan']
after encoding = [ 3  4 12  1  6 10  0  9 14  8 13  2  7  5 11]


In [ ]:
accounts_train_save.head()

,credit_type,loan_amount,amount_overdue,uid,decode_payment_hist_string,loan_period,NAME_CONTRACT_TYPE,TARGET
0,3,272745.000,0.0,AAA09044550,10.0,518 days,Cash loans,0
1,3,4500.000,0.0,AAA09044550,29.0,504 days,Cash loans,0
2,4,80996.445,0.0,AAA10545297,0.0,672 days,Cash loans,0
3,3,43771.500,0.0,AAA14112888,0.0,92 days,Cash loans,0
4,4,10480.500,0.0,AAA20326915,467.0,2791 days,Cash loans,0


In [ ]:
accounts_train.to_pickle(BASE_PATH + '/data/train/accounts_clean_train.pkl')

## Flag 

In [ ]:
flag_train = pd.read_csv(BASE_PATH + '/data/train/train_flag.csv')
print(flag_train.shape, flag_train.columns, flag_train.uid.nunique())
flag_train.head(5)

(261383, 3) Index(['uid', 'NAME_CONTRACT_TYPE', 'TARGET'], dtype='object') 261383


,uid,NAME_CONTRACT_TYPE,TARGET
0,XDA69787158,Cash loans,0
1,BSE47789733,Cash loans,0
2,NTJ92213825,Cash loans,0
3,TCQ47571695,Cash loans,0
4,WJZ68772744,Cash loans,0


In [ ]:
print(flag_train.TARGET.value_counts())
flag_train.NAME_CONTRACT_TYPE.unique()

0    240326
1     21057
Name: TARGET, dtype: int64


array(['Cash loans', 'Revolving loans'], dtype=object)